# Messaging system with *RabbitMQ* and CSharp

Na zajęciach zostaną przedstawione następujące zagadnienia:

- terminologia związana z przetwarzaniem kolejkowym,
- opis usługi *RabbitMQ*, w tym uruchomienie, wysyłanie i odbieranie wiadomości,
- podstawy *Event Sourcing*.

## Wprowadzenie

Kolejka jako struktura danych pod względem konstrukcji jest niezwykle prosta. Należy wyszczególnić następujące operacje na kolejce:
- dodanie obiektu do kolejki,
- pobranie elementu z kolejki oraz jego obsługa (co powoduje jego usunięcie).

Pod względem algorytmów i struktur danych kolejki mogą być typu:
 - *LIFO* (*last in first out*) z kolejki pobierany jest najstarszy element.
 - *FIFO* (*first in first out*) z kolejki pobierany jest najmłodszy element.
 - *Priority Queue* z kolejki pobierany jest element o najwyższym priorytecie, co wymaga sortowania w trakcie dodawania lub pobierania elementu.

Implementacja kolejki jako systemu do obsługi zdarzeń jest efektywne, gdyż nie ma przestojów w procesowaniu danych. Po zakończeniu jednego zadania proces bierze następne, co jest niezwykle efektywne w szczególności tam, gdzie czas trwania przetwarzania jest nieznany. W nomenklaturze systemów zdarzeniowych implementacje kolejek i całe oprogramowanie wokół nazywamy brokerem wiadomości, gdzie wiadomością w naszym przypadku jest zdarzenie. Za operację dodawania odpowiedzialny jest publisher, a za pobieranie danych subskrybent. W zależności od liczby publisherów mamy:
- `queue` - klasyczna kolejka z wieloma źródłami zdarzeń,
- `event bus` - zawsze tylko jedno źródło zdarzeń.

Kanałem komunikacyjnym jest logiczne powiązanie producenta, brokera i konsumenta wiadomości. *Router* (*Exchange*) odpowiada za wysłanie komunikatu do konkretnej kolejki na podstawie treści zawartej w wiadomości. Poniżej znajduje się diagram prezentujący kolejne komponenty komunikacji w *RabbitMQ*.

<center>
<img src='data:image/svg+xml;base64,
PHN2ZyB4bWxucz0iaHR0cDovL3d3dy53My5vcmcvMjAwMC9zdmciIHhtbG5zOnhsaW5rPSJodHRwOi8vd3d3LnczLm9yZy8xOTk5L3hsaW5rIiB2ZXJzaW9uPSIxLjEiIHdpZHRoPSI4MjJweCIgaGVpZ2h0PSIyNjdweCIgdmlld0JveD0iLTAuNSAtMC41IDgyMiAyNjciIGNvbnRlbnQ9IiZsdDtteGZpbGUgaG9zdD0mcXVvdDthcHAuZGlhZ3JhbXMubmV0JnF1b3Q7IG1vZGlmaWVkPSZxdW90OzIwMjEtMDQtMjdUMTU6NTE6MjIuOTY5WiZxdW90OyBhZ2VudD0mcXVvdDs1LjAgKFdpbmRvd3MpJnF1b3Q7IGV0YWc9JnF1b3Q7Zi1NSkNLREE0Y1gwOE8zUGM1dnImcXVvdDsgdmVyc2lvbj0mcXVvdDsxNC42LjYmcXVvdDsgdHlwZT0mcXVvdDtkZXZpY2UmcXVvdDsmZ3Q7Jmx0O2RpYWdyYW0gaWQ9JnF1b3Q7VTFxMnducnQ4YTBybWg3aHByUEomcXVvdDsgbmFtZT0mcXVvdDtQYWdlLTEmcXVvdDsmZ3Q7N1psZGI1c3dGSVovRFplYkFBTkpMaHVhZGFwVXFWczE3ZHJGRGxneWRtUk1RdkxyWjRyNU5FMWJOUjFKVlVXS3pHdjdHSjV6c0gyTUJjSzB1QkZ3azl4eGhLbmwycWl3d0xYbHVyN25xdjlTMkZmQ3pBc3FJUllFVlpMVENnL2tnTFZvYXpVbkNHZTlocEp6S3NtbUwwYWNNUnpKbmdhRjRMdCtzelduL1ZFM01NYUc4QkJCYXFwL0NaSkpwYzU5dTlWL1loSW45Y2lPcld0U1dEZldRcFpBeEhjZENhd3NFQXJPWlZWS2l4RFRrbDNOcGVyMzQ1bmE1c1lFWnZJMUhWaTIrYk4rdkxvdERvZmI0T2J1R3N4WDZUZG5VWm5aUXBycko5WjNLL2MxQXNGemhuQnB4YmJBY3BjUWlSODJNQ3ByZDhyblNrdGtTdFdWbzRwclFtbklLUmZxbW5HbUdpMzFDRmhJWER4NzcwNURSRVVTNWltV1lxK2E2QTVlb0NIcUtIS0FYMTN2V3A4NG5tNlRkUHpoQnJvaDFIRVFON1piVktxZ2FiMkYzUHdpeURYaGQwYmtSc0FGVkEyN1JHU3JpbkZadkJjYzVSRVdkWlVhcVZON2hMVHozMG52QjZTN1pOMFJzblVzbnh5cyt4cXdJV2Rabmw0RVdOMWhadmREZUhyUW5nRjZWVVFKWkdvcGNXM25IQm02WURBTitGTXpERjRUckw5eW5HdW1seEt0M3V6Y1NNL2VRdHE5WE5MVHp3dGpXNEpuU1lPNjdsRzBsQytXL1dKcTl1Wkd0ak1uSDl1YW5jMmNQSDM4MWtuWE9FUndFUkFuRDBUSDNCMFk0RlF1dUNtTGE0cUxxekpMVlRBd1E3cDRIVkdZWlNRYThPTk0xdndzRi9oaCtUdkdGU01qdFgyUmFvZWFQd0t0MWdTbVVKSnQzL3dZU1QzQ1BTZHE0TVpwVGNKUk84MWJmSjhGZlNzWnowV0VkY2R1V2p1dzFad1VITEVsb1lpeE5Hdzl1YmQ1K0hmc3U4M1g1c3ZqZlMrQitlazg3dm5UZTN3c2gvM3llTmRMd1FrOUhwekJPMjRtMTc5NUxzczgycVo0V3g1NkR2eXYxalQ1NGdvSUtZbFpHUTNLYWNvV1dKYUxJWWtndmRJVktVR290RGk2dXZiUGwwNnhtZzYyZFdCbXJxYWpoMFVmZHFUaEc5aWY5cy9acDhEdE8rZUcyOHdWdytwZ25YRDJLWkE3dzhWamN1Um0wdmpKa0FmRExmckhJVmVYN1FlVmF1cHZ2MHFCMVQ4PSZsdDsvZGlhZ3JhbSZndDsmbHQ7L214ZmlsZSZndDsiPjxkZWZzLz48Zz48cmVjdCB4PSI0NjAiIHk9IjAiIHdpZHRoPSIxNDAiIGhlaWdodD0iMjY1IiBmaWxsPSJub25lIiBzdHJva2U9IiMwMDAwMDAiIHBvaW50ZXItZXZlbnRzPSJhbGwiLz48cmVjdCB4PSIyMjAiIHk9IjAiIHdpZHRoPSIxNDAiIGhlaWdodD0iMjY1IiBmaWxsPSJub25lIiBzdHJva2U9IiMwMDAwMDAiIHBvaW50ZXItZXZlbnRzPSJhbGwiLz48cmVjdCB4PSIwIiB5PSI4NSIgd2lkdGg9IjEyMCIgaGVpZ2h0PSI2MCIgcng9IjkiIHJ5PSI5IiBmaWxsPSJub25lIiBzdHJva2U9IiMwMDAwMDAiIHBvaW50ZXItZXZlbnRzPSJhbGwiLz48ZyB0cmFuc2Zvcm09InRyYW5zbGF0ZSgtMC41IC0wLjUpIj48c3dpdGNoPjxmb3JlaWduT2JqZWN0IHN0eWxlPSJvdmVyZmxvdzogdmlzaWJsZTsgdGV4dC1hbGlnbjogbGVmdDsiIHBvaW50ZXItZXZlbnRzPSJub25lIiB3aWR0aD0iMTAwJSIgaGVpZ2h0PSIxMDAlIiByZXF1aXJlZEZlYXR1cmVzPSJodHRwOi8vd3d3LnczLm9yZy9UUi9TVkcxMS9mZWF0dXJlI0V4dGVuc2liaWxpdHkiPjxkaXYgeG1sbnM9Imh0dHA6Ly93d3cudzMub3JnLzE5OTkveGh0bWwiIHN0eWxlPSJkaXNwbGF5OiBmbGV4OyBhbGlnbi1pdGVtczogdW5zYWZlIGNlbnRlcjsganVzdGlmeS1jb250ZW50OiB1bnNhZmUgY2VudGVyOyB3aWR0aDogMTE4cHg7IGhlaWdodDogMXB4OyBwYWRkaW5nLXRvcDogMTE1cHg7IG1hcmdpbi1sZWZ0OiAxcHg7Ij48ZGl2IHN0eWxlPSJib3gtc2l6aW5nOiBib3JkZXItYm94OyBmb250LXNpemU6IDA7IHRleHQtYWxpZ246IGNlbnRlcjsgIj48ZGl2IHN0eWxlPSJkaXNwbGF5OiBpbmxpbmUtYmxvY2s7IGZvbnQtc2l6ZTogMTJweDsgZm9udC1mYW1pbHk6IEhlbHZldGljYTsgY29sb3I6ICMwMDAwMDA7IGxpbmUtaGVpZ2h0OiAxLjI7IHBvaW50ZXItZXZlbnRzOiBhbGw7IHdoaXRlLXNwYWNlOiBub3JtYWw7IHdvcmQtd3JhcDogbm9ybWFsOyAiPjxkaXY+UHJvZHVjZXI8L2Rpdj48L2Rpdj48L2Rpdj48L2Rpdj48L2ZvcmVpZ25PYmplY3Q+PHRleHQgeD0iNjAiIHk9IjExOSIgZmlsbD0iIzAwMDAwMCIgZm9udC1mYW1pbHk9IkhlbHZldGljYSIgZm9udC1zaXplPSIxMnB4IiB0ZXh0LWFuY2hvcj0ibWlkZGxlIj5Qcm9kdWNlcjwvdGV4dD48L3N3aXRjaD48L2c+PHJlY3QgeD0iNzAwIiB5PSI4NSIgd2lkdGg9IjEyMCIgaGVpZ2h0PSI2MCIgcng9IjkiIHJ5PSI5IiBmaWxsPSJub25lIiBzdHJva2U9IiMwMDAwMDAiIHBvaW50ZXItZXZlbnRzPSJhbGwiLz48ZyB0cmFuc2Zvcm09InRyYW5zbGF0ZSgtMC41IC0wLjUpIj48c3dpdGNoPjxmb3JlaWduT2JqZWN0IHN0eWxlPSJvdmVyZmxvdzogdmlzaWJsZTsgdGV4dC1hbGlnbjogbGVmdDsiIHBvaW50ZXItZXZlbnRzPSJub25lIiB3aWR0aD0iMTAwJSIgaGVpZ2h0PSIxMDAlIiByZXF1aXJlZEZlYXR1cmVzPSJodHRwOi8vd3d3LnczLm9yZy9UUi9TVkcxMS9mZWF0dXJlI0V4dGVuc2liaWxpdHkiPjxkaXYgeG1sbnM9Imh0dHA6Ly93d3cudzMub3JnLzE5OTkveGh0bWwiIHN0eWxlPSJkaXNwbGF5OiBmbGV4OyBhbGlnbi1pdGVtczogdW5zYWZlIGNlbnRlcjsganVzdGlmeS1jb250ZW50OiB1bnNhZmUgY2VudGVyOyB3aWR0aDogMTE4cHg7IGhlaWdodDogMXB4OyBwYWRkaW5nLXRvcDogMTE1cHg7IG1hcmdpbi1sZWZ0OiA3MDFweDsiPjxkaXYgc3R5bGU9ImJveC1zaXppbmc6IGJvcmRlci1ib3g7IGZvbnQtc2l6ZTogMDsgdGV4dC1hbGlnbjogY2VudGVyOyAiPjxkaXYgc3R5bGU9ImRpc3BsYXk6IGlubGluZS1ibG9jazsgZm9udC1zaXplOiAxMnB4OyBmb250LWZhbWlseTogSGVsdmV0aWNhOyBjb2xvcjogIzAwMDAwMDsgbGluZS1oZWlnaHQ6IDEuMjsgcG9pbnRlci1ldmVudHM6IGFsbDsgd2hpdGUtc3BhY2U6IG5vcm1hbDsgd29yZC13cmFwOiBub3JtYWw7ICI+PGRpdj5Db25zdW1lcjwvZGl2PjwvZGl2PjwvZGl2PjwvZGl2PjwvZm9yZWlnbk9iamVjdD48dGV4dCB4PSI3NjAiIHk9IjExOSIgZmlsbD0iIzAwMDAwMCIgZm9udC1mYW1pbHk9IkhlbHZldGljYSIgZm9udC1zaXplPSIxMnB4IiB0ZXh0LWFuY2hvcj0ibWlkZGxlIj5Db25zdW1lcjwvdGV4dD48L3N3aXRjaD48L2c+PHJlY3QgeD0iMjMwIiB5PSIxNSIgd2lkdGg9IjEyMCIgaGVpZ2h0PSI2MCIgcng9IjkiIHJ5PSI5IiBmaWxsPSJub25lIiBzdHJva2U9IiMwMDAwMDAiIHBvaW50ZXItZXZlbnRzPSJhbGwiLz48ZyB0cmFuc2Zvcm09InRyYW5zbGF0ZSgtMC41IC0wLjUpIj48c3dpdGNoPjxmb3JlaWduT2JqZWN0IHN0eWxlPSJvdmVyZmxvdzogdmlzaWJsZTsgdGV4dC1hbGlnbjogbGVmdDsiIHBvaW50ZXItZXZlbnRzPSJub25lIiB3aWR0aD0iMTAwJSIgaGVpZ2h0PSIxMDAlIiByZXF1aXJlZEZlYXR1cmVzPSJodHRwOi8vd3d3LnczLm9yZy9UUi9TVkcxMS9mZWF0dXJlI0V4dGVuc2liaWxpdHkiPjxkaXYgeG1sbnM9Imh0dHA6Ly93d3cudzMub3JnLzE5OTkveGh0bWwiIHN0eWxlPSJkaXNwbGF5OiBmbGV4OyBhbGlnbi1pdGVtczogdW5zYWZlIGNlbnRlcjsganVzdGlmeS1jb250ZW50OiB1bnNhZmUgY2VudGVyOyB3aWR0aDogMTE4cHg7IGhlaWdodDogMXB4OyBwYWRkaW5nLXRvcDogNDVweDsgbWFyZ2luLWxlZnQ6IDIzMXB4OyI+PGRpdiBzdHlsZT0iYm94LXNpemluZzogYm9yZGVyLWJveDsgZm9udC1zaXplOiAwOyB0ZXh0LWFsaWduOiBjZW50ZXI7ICI+PGRpdiBzdHlsZT0iZGlzcGxheTogaW5saW5lLWJsb2NrOyBmb250LXNpemU6IDEycHg7IGZvbnQtZmFtaWx5OiBIZWx2ZXRpY2E7IGNvbG9yOiAjMDAwMDAwOyBsaW5lLWhlaWdodDogMS4yOyBwb2ludGVyLWV2ZW50czogYWxsOyB3aGl0ZS1zcGFjZTogbm9ybWFsOyB3b3JkLXdyYXA6IG5vcm1hbDsgIj5FeGNoYW5nZSAxPC9kaXY+PC9kaXY+PC9kaXY+PC9mb3JlaWduT2JqZWN0Pjx0ZXh0IHg9IjI5MCIgeT0iNDkiIGZpbGw9IiMwMDAwMDAiIGZvbnQtZmFtaWx5PSJIZWx2ZXRpY2EiIGZvbnQtc2l6ZT0iMTJweCIgdGV4dC1hbmNob3I9Im1pZGRsZSI+RXhjaGFuZ2UgMTwvdGV4dD48L3N3aXRjaD48L2c+PHJlY3QgeD0iNDcwIiB5PSIxNSIgd2lkdGg9IjEyMCIgaGVpZ2h0PSI2MCIgcng9IjkiIHJ5PSI5IiBmaWxsPSJub25lIiBzdHJva2U9IiMwMDAwMDAiIHBvaW50ZXItZXZlbnRzPSJhbGwiLz48ZyB0cmFuc2Zvcm09InRyYW5zbGF0ZSgtMC41IC0wLjUpIj48c3dpdGNoPjxmb3JlaWduT2JqZWN0IHN0eWxlPSJvdmVyZmxvdzogdmlzaWJsZTsgdGV4dC1hbGlnbjogbGVmdDsiIHBvaW50ZXItZXZlbnRzPSJub25lIiB3aWR0aD0iMTAwJSIgaGVpZ2h0PSIxMDAlIiByZXF1aXJlZEZlYXR1cmVzPSJodHRwOi8vd3d3LnczLm9yZy9UUi9TVkcxMS9mZWF0dXJlI0V4dGVuc2liaWxpdHkiPjxkaXYgeG1sbnM9Imh0dHA6Ly93d3cudzMub3JnLzE5OTkveGh0bWwiIHN0eWxlPSJkaXNwbGF5OiBmbGV4OyBhbGlnbi1pdGVtczogdW5zYWZlIGNlbnRlcjsganVzdGlmeS1jb250ZW50OiB1bnNhZmUgY2VudGVyOyB3aWR0aDogMTE4cHg7IGhlaWdodDogMXB4OyBwYWRkaW5nLXRvcDogNDVweDsgbWFyZ2luLWxlZnQ6IDQ3MXB4OyI+PGRpdiBzdHlsZT0iYm94LXNpemluZzogYm9yZGVyLWJveDsgZm9udC1zaXplOiAwOyB0ZXh0LWFsaWduOiBjZW50ZXI7ICI+PGRpdiBzdHlsZT0iZGlzcGxheTogaW5saW5lLWJsb2NrOyBmb250LXNpemU6IDEycHg7IGZvbnQtZmFtaWx5OiBIZWx2ZXRpY2E7IGNvbG9yOiAjMDAwMDAwOyBsaW5lLWhlaWdodDogMS4yOyBwb2ludGVyLWV2ZW50czogYWxsOyB3aGl0ZS1zcGFjZTogbm9ybWFsOyB3b3JkLXdyYXA6IG5vcm1hbDsgIj48ZGl2PlF1ZXVlIDE8L2Rpdj48L2Rpdj48L2Rpdj48L2Rpdj48L2ZvcmVpZ25PYmplY3Q+PHRleHQgeD0iNTMwIiB5PSI0OSIgZmlsbD0iIzAwMDAwMCIgZm9udC1mYW1pbHk9IkhlbHZldGljYSIgZm9udC1zaXplPSIxMnB4IiB0ZXh0LWFuY2hvcj0ibWlkZGxlIj5RdWV1ZSAxPC90ZXh0Pjwvc3dpdGNoPjwvZz48cmVjdCB4PSI0NzAiIHk9Ijg1IiB3aWR0aD0iMTIwIiBoZWlnaHQ9IjYwIiByeD0iOSIgcnk9IjkiIGZpbGw9Im5vbmUiIHN0cm9rZT0iIzAwMDAwMCIgcG9pbnRlci1ldmVudHM9ImFsbCIvPjxnIHRyYW5zZm9ybT0idHJhbnNsYXRlKC0wLjUgLTAuNSkiPjxzd2l0Y2g+PGZvcmVpZ25PYmplY3Qgc3R5bGU9Im92ZXJmbG93OiB2aXNpYmxlOyB0ZXh0LWFsaWduOiBsZWZ0OyIgcG9pbnRlci1ldmVudHM9Im5vbmUiIHdpZHRoPSIxMDAlIiBoZWlnaHQ9IjEwMCUiIHJlcXVpcmVkRmVhdHVyZXM9Imh0dHA6Ly93d3cudzMub3JnL1RSL1NWRzExL2ZlYXR1cmUjRXh0ZW5zaWJpbGl0eSI+PGRpdiB4bWxucz0iaHR0cDovL3d3dy53My5vcmcvMTk5OS94aHRtbCIgc3R5bGU9ImRpc3BsYXk6IGZsZXg7IGFsaWduLWl0ZW1zOiB1bnNhZmUgY2VudGVyOyBqdXN0aWZ5LWNvbnRlbnQ6IHVuc2FmZSBjZW50ZXI7IHdpZHRoOiAxMThweDsgaGVpZ2h0OiAxcHg7IHBhZGRpbmctdG9wOiAxMTVweDsgbWFyZ2luLWxlZnQ6IDQ3MXB4OyI+PGRpdiBzdHlsZT0iYm94LXNpemluZzogYm9yZGVyLWJveDsgZm9udC1zaXplOiAwOyB0ZXh0LWFsaWduOiBjZW50ZXI7ICI+PGRpdiBzdHlsZT0iZGlzcGxheTogaW5saW5lLWJsb2NrOyBmb250LXNpemU6IDEycHg7IGZvbnQtZmFtaWx5OiBIZWx2ZXRpY2E7IGNvbG9yOiAjMDAwMDAwOyBsaW5lLWhlaWdodDogMS4yOyBwb2ludGVyLWV2ZW50czogYWxsOyB3aGl0ZS1zcGFjZTogbm9ybWFsOyB3b3JkLXdyYXA6IG5vcm1hbDsgIj48ZGl2PlF1ZXVlIDI8L2Rpdj48L2Rpdj48L2Rpdj48L2Rpdj48L2ZvcmVpZ25PYmplY3Q+PHRleHQgeD0iNTMwIiB5PSIxMTkiIGZpbGw9IiMwMDAwMDAiIGZvbnQtZmFtaWx5PSJIZWx2ZXRpY2EiIGZvbnQtc2l6ZT0iMTJweCIgdGV4dC1hbmNob3I9Im1pZGRsZSI+UXVldWUgMjwvdGV4dD48L3N3aXRjaD48L2c+PHJlY3QgeD0iNDcwIiB5PSIxNTUiIHdpZHRoPSIxMjAiIGhlaWdodD0iNjAiIHJ4PSI5IiByeT0iOSIgZmlsbD0ibm9uZSIgc3Ryb2tlPSIjMDAwMDAwIiBwb2ludGVyLWV2ZW50cz0iYWxsIi8+PGcgdHJhbnNmb3JtPSJ0cmFuc2xhdGUoLTAuNSAtMC41KSI+PHN3aXRjaD48Zm9yZWlnbk9iamVjdCBzdHlsZT0ib3ZlcmZsb3c6IHZpc2libGU7IHRleHQtYWxpZ246IGxlZnQ7IiBwb2ludGVyLWV2ZW50cz0ibm9uZSIgd2lkdGg9IjEwMCUiIGhlaWdodD0iMTAwJSIgcmVxdWlyZWRGZWF0dXJlcz0iaHR0cDovL3d3dy53My5vcmcvVFIvU1ZHMTEvZmVhdHVyZSNFeHRlbnNpYmlsaXR5Ij48ZGl2IHhtbG5zPSJodHRwOi8vd3d3LnczLm9yZy8xOTk5L3hodG1sIiBzdHlsZT0iZGlzcGxheTogZmxleDsgYWxpZ24taXRlbXM6IHVuc2FmZSBjZW50ZXI7IGp1c3RpZnktY29udGVudDogdW5zYWZlIGNlbnRlcjsgd2lkdGg6IDExOHB4OyBoZWlnaHQ6IDFweDsgcGFkZGluZy10b3A6IDE4NXB4OyBtYXJnaW4tbGVmdDogNDcxcHg7Ij48ZGl2IHN0eWxlPSJib3gtc2l6aW5nOiBib3JkZXItYm94OyBmb250LXNpemU6IDA7IHRleHQtYWxpZ246IGNlbnRlcjsgIj48ZGl2IHN0eWxlPSJkaXNwbGF5OiBpbmxpbmUtYmxvY2s7IGZvbnQtc2l6ZTogMTJweDsgZm9udC1mYW1pbHk6IEhlbHZldGljYTsgY29sb3I6ICMwMDAwMDA7IGxpbmUtaGVpZ2h0OiAxLjI7IHBvaW50ZXItZXZlbnRzOiBhbGw7IHdoaXRlLXNwYWNlOiBub3JtYWw7IHdvcmQtd3JhcDogbm9ybWFsOyAiPjxkaXY+UXVldWUgMzxiciAvPjwvZGl2PjwvZGl2PjwvZGl2PjwvZGl2PjwvZm9yZWlnbk9iamVjdD48dGV4dCB4PSI1MzAiIHk9IjE4OSIgZmlsbD0iIzAwMDAwMCIgZm9udC1mYW1pbHk9IkhlbHZldGljYSIgZm9udC1zaXplPSIxMnB4IiB0ZXh0LWFuY2hvcj0ibWlkZGxlIj5RdWV1ZSAzJiN4YTs8L3RleHQ+PC9zd2l0Y2g+PC9nPjxyZWN0IHg9IjIzMCIgeT0iODUiIHdpZHRoPSIxMjAiIGhlaWdodD0iNjAiIHJ4PSI5IiByeT0iOSIgZmlsbD0ibm9uZSIgc3Ryb2tlPSIjMDAwMDAwIiBwb2ludGVyLWV2ZW50cz0iYWxsIi8+PGcgdHJhbnNmb3JtPSJ0cmFuc2xhdGUoLTAuNSAtMC41KSI+PHN3aXRjaD48Zm9yZWlnbk9iamVjdCBzdHlsZT0ib3ZlcmZsb3c6IHZpc2libGU7IHRleHQtYWxpZ246IGxlZnQ7IiBwb2ludGVyLWV2ZW50cz0ibm9uZSIgd2lkdGg9IjEwMCUiIGhlaWdodD0iMTAwJSIgcmVxdWlyZWRGZWF0dXJlcz0iaHR0cDovL3d3dy53My5vcmcvVFIvU1ZHMTEvZmVhdHVyZSNFeHRlbnNpYmlsaXR5Ij48ZGl2IHhtbG5zPSJodHRwOi8vd3d3LnczLm9yZy8xOTk5L3hodG1sIiBzdHlsZT0iZGlzcGxheTogZmxleDsgYWxpZ24taXRlbXM6IHVuc2FmZSBjZW50ZXI7IGp1c3RpZnktY29udGVudDogdW5zYWZlIGNlbnRlcjsgd2lkdGg6IDExOHB4OyBoZWlnaHQ6IDFweDsgcGFkZGluZy10b3A6IDExNXB4OyBtYXJnaW4tbGVmdDogMjMxcHg7Ij48ZGl2IHN0eWxlPSJib3gtc2l6aW5nOiBib3JkZXItYm94OyBmb250LXNpemU6IDA7IHRleHQtYWxpZ246IGNlbnRlcjsgIj48ZGl2IHN0eWxlPSJkaXNwbGF5OiBpbmxpbmUtYmxvY2s7IGZvbnQtc2l6ZTogMTJweDsgZm9udC1mYW1pbHk6IEhlbHZldGljYTsgY29sb3I6ICMwMDAwMDA7IGxpbmUtaGVpZ2h0OiAxLjI7IHBvaW50ZXItZXZlbnRzOiBhbGw7IHdoaXRlLXNwYWNlOiBub3JtYWw7IHdvcmQtd3JhcDogbm9ybWFsOyAiPkV4Y2hhbmdlIDI8L2Rpdj48L2Rpdj48L2Rpdj48L2ZvcmVpZ25PYmplY3Q+PHRleHQgeD0iMjkwIiB5PSIxMTkiIGZpbGw9IiMwMDAwMDAiIGZvbnQtZmFtaWx5PSJIZWx2ZXRpY2EiIGZvbnQtc2l6ZT0iMTJweCIgdGV4dC1hbmNob3I9Im1pZGRsZSI+RXhjaGFuZ2UgMjwvdGV4dD48L3N3aXRjaD48L2c+PHJlY3QgeD0iMjMwIiB5PSIxNTUiIHdpZHRoPSIxMjAiIGhlaWdodD0iNjAiIHJ4PSI5IiByeT0iOSIgZmlsbD0ibm9uZSIgc3Ryb2tlPSIjMDAwMDAwIiBwb2ludGVyLWV2ZW50cz0iYWxsIi8+PGcgdHJhbnNmb3JtPSJ0cmFuc2xhdGUoLTAuNSAtMC41KSI+PHN3aXRjaD48Zm9yZWlnbk9iamVjdCBzdHlsZT0ib3ZlcmZsb3c6IHZpc2libGU7IHRleHQtYWxpZ246IGxlZnQ7IiBwb2ludGVyLWV2ZW50cz0ibm9uZSIgd2lkdGg9IjEwMCUiIGhlaWdodD0iMTAwJSIgcmVxdWlyZWRGZWF0dXJlcz0iaHR0cDovL3d3dy53My5vcmcvVFIvU1ZHMTEvZmVhdHVyZSNFeHRlbnNpYmlsaXR5Ij48ZGl2IHhtbG5zPSJodHRwOi8vd3d3LnczLm9yZy8xOTk5L3hodG1sIiBzdHlsZT0iZGlzcGxheTogZmxleDsgYWxpZ24taXRlbXM6IHVuc2FmZSBjZW50ZXI7IGp1c3RpZnktY29udGVudDogdW5zYWZlIGNlbnRlcjsgd2lkdGg6IDExOHB4OyBoZWlnaHQ6IDFweDsgcGFkZGluZy10b3A6IDE4NXB4OyBtYXJnaW4tbGVmdDogMjMxcHg7Ij48ZGl2IHN0eWxlPSJib3gtc2l6aW5nOiBib3JkZXItYm94OyBmb250LXNpemU6IDA7IHRleHQtYWxpZ246IGNlbnRlcjsgIj48ZGl2IHN0eWxlPSJkaXNwbGF5OiBpbmxpbmUtYmxvY2s7IGZvbnQtc2l6ZTogMTJweDsgZm9udC1mYW1pbHk6IEhlbHZldGljYTsgY29sb3I6ICMwMDAwMDA7IGxpbmUtaGVpZ2h0OiAxLjI7IHBvaW50ZXItZXZlbnRzOiBhbGw7IHdoaXRlLXNwYWNlOiBub3JtYWw7IHdvcmQtd3JhcDogbm9ybWFsOyAiPkV4Y2hhbmdlIDM8L2Rpdj48L2Rpdj48L2Rpdj48L2ZvcmVpZ25PYmplY3Q+PHRleHQgeD0iMjkwIiB5PSIxODkiIGZpbGw9IiMwMDAwMDAiIGZvbnQtZmFtaWx5PSJIZWx2ZXRpY2EiIGZvbnQtc2l6ZT0iMTJweCIgdGV4dC1hbmNob3I9Im1pZGRsZSI+RXhjaGFuZ2UgMzwvdGV4dD48L3N3aXRjaD48L2c+PHBhdGggZD0iTSAxNDAuNSAxMTkuNzYgTCAxNDAuNSAxMDkuNzYgTCAxOTAuNSAxMDkuNzYgTCAxOTAuNSA5OS4yNiBMIDIwOS41IDExNC43NiBMIDE5MC41IDEzMC4yNiBMIDE5MC41IDExOS43NiBaIiBmaWxsPSJub25lIiBzdHJva2U9IiMwMDAwMDAiIHN0cm9rZS1saW5lam9pbj0icm91bmQiIHN0cm9rZS1taXRlcmxpbWl0PSIxMCIgcG9pbnRlci1ldmVudHM9ImFsbCIvPjxwYXRoIGQ9Ik0gMzgwLjUgMTE5Ljc2IEwgMzgwLjUgMTA5Ljc2IEwgNDMwLjUgMTA5Ljc2IEwgNDMwLjUgOTkuMjYgTCA0NDkuNSAxMTQuNzYgTCA0MzAuNSAxMzAuMjYgTCA0MzAuNSAxMTkuNzYgWiIgZmlsbD0ibm9uZSIgc3Ryb2tlPSIjMDAwMDAwIiBzdHJva2UtbGluZWpvaW49InJvdW5kIiBzdHJva2UtbWl0ZXJsaW1pdD0iMTAiIHBvaW50ZXItZXZlbnRzPSJhbGwiLz48cGF0aCBkPSJNIDY3OS41IDEwOS43NiBMIDY3OS41IDExOS43NiBMIDYyOS41IDExOS43NiBMIDYyOS41IDEzMC4yNiBMIDYxMC41IDExNC43NiBMIDYyOS41IDk5LjI2IEwgNjI5LjUgMTA5Ljc2IFoiIGZpbGw9Im5vbmUiIHN0cm9rZT0iIzAwMDAwMCIgc3Ryb2tlLWxpbmVqb2luPSJyb3VuZCIgc3Ryb2tlLW1pdGVybGltaXQ9IjEwIiBwb2ludGVyLWV2ZW50cz0iYWxsIi8+PHJlY3QgeD0iMjcwIiB5PSIyMzUiIHdpZHRoPSI0MCIgaGVpZ2h0PSIyMCIgZmlsbD0ibm9uZSIgc3Ryb2tlPSJub25lIiBwb2ludGVyLWV2ZW50cz0iYWxsIi8+PGcgdHJhbnNmb3JtPSJ0cmFuc2xhdGUoLTAuNSAtMC41KSI+PHN3aXRjaD48Zm9yZWlnbk9iamVjdCBzdHlsZT0ib3ZlcmZsb3c6IHZpc2libGU7IHRleHQtYWxpZ246IGxlZnQ7IiBwb2ludGVyLWV2ZW50cz0ibm9uZSIgd2lkdGg9IjEwMCUiIGhlaWdodD0iMTAwJSIgcmVxdWlyZWRGZWF0dXJlcz0iaHR0cDovL3d3dy53My5vcmcvVFIvU1ZHMTEvZmVhdHVyZSNFeHRlbnNpYmlsaXR5Ij48ZGl2IHhtbG5zPSJodHRwOi8vd3d3LnczLm9yZy8xOTk5L3hodG1sIiBzdHlsZT0iZGlzcGxheTogZmxleDsgYWxpZ24taXRlbXM6IHVuc2FmZSBjZW50ZXI7IGp1c3RpZnktY29udGVudDogdW5zYWZlIGNlbnRlcjsgd2lkdGg6IDM4cHg7IGhlaWdodDogMXB4OyBwYWRkaW5nLXRvcDogMjQ1cHg7IG1hcmdpbi1sZWZ0OiAyNzFweDsiPjxkaXYgc3R5bGU9ImJveC1zaXppbmc6IGJvcmRlci1ib3g7IGZvbnQtc2l6ZTogMDsgdGV4dC1hbGlnbjogY2VudGVyOyAiPjxkaXYgc3R5bGU9ImRpc3BsYXk6IGlubGluZS1ibG9jazsgZm9udC1zaXplOiAxMnB4OyBmb250LWZhbWlseTogSGVsdmV0aWNhOyBjb2xvcjogIzAwMDAwMDsgbGluZS1oZWlnaHQ6IDEuMjsgcG9pbnRlci1ldmVudHM6IGFsbDsgd2hpdGUtc3BhY2U6IG5vcm1hbDsgd29yZC13cmFwOiBub3JtYWw7ICI+Um91dGVyIGxldmVsPC9kaXY+PC9kaXY+PC9kaXY+PC9mb3JlaWduT2JqZWN0Pjx0ZXh0IHg9IjI5MCIgeT0iMjQ5IiBmaWxsPSIjMDAwMDAwIiBmb250LWZhbWlseT0iSGVsdmV0aWNhIiBmb250LXNpemU9IjEycHgiIHRleHQtYW5jaG9yPSJtaWRkbGUiPlJvdXRlci4uLjwvdGV4dD48L3N3aXRjaD48L2c+PHJlY3QgeD0iNTEwIiB5PSIyMzUiIHdpZHRoPSI0MCIgaGVpZ2h0PSIyMCIgZmlsbD0ibm9uZSIgc3Ryb2tlPSJub25lIiBwb2ludGVyLWV2ZW50cz0iYWxsIi8+PGcgdHJhbnNmb3JtPSJ0cmFuc2xhdGUoLTAuNSAtMC41KSI+PHN3aXRjaD48Zm9yZWlnbk9iamVjdCBzdHlsZT0ib3ZlcmZsb3c6IHZpc2libGU7IHRleHQtYWxpZ246IGxlZnQ7IiBwb2ludGVyLWV2ZW50cz0ibm9uZSIgd2lkdGg9IjEwMCUiIGhlaWdodD0iMTAwJSIgcmVxdWlyZWRGZWF0dXJlcz0iaHR0cDovL3d3dy53My5vcmcvVFIvU1ZHMTEvZmVhdHVyZSNFeHRlbnNpYmlsaXR5Ij48ZGl2IHhtbG5zPSJodHRwOi8vd3d3LnczLm9yZy8xOTk5L3hodG1sIiBzdHlsZT0iZGlzcGxheTogZmxleDsgYWxpZ24taXRlbXM6IHVuc2FmZSBjZW50ZXI7IGp1c3RpZnktY29udGVudDogdW5zYWZlIGNlbnRlcjsgd2lkdGg6IDM4cHg7IGhlaWdodDogMXB4OyBwYWRkaW5nLXRvcDogMjQ1cHg7IG1hcmdpbi1sZWZ0OiA1MTFweDsiPjxkaXYgc3R5bGU9ImJveC1zaXppbmc6IGJvcmRlci1ib3g7IGZvbnQtc2l6ZTogMDsgdGV4dC1hbGlnbjogY2VudGVyOyAiPjxkaXYgc3R5bGU9ImRpc3BsYXk6IGlubGluZS1ibG9jazsgZm9udC1zaXplOiAxMnB4OyBmb250LWZhbWlseTogSGVsdmV0aWNhOyBjb2xvcjogIzAwMDAwMDsgbGluZS1oZWlnaHQ6IDEuMjsgcG9pbnRlci1ldmVudHM6IGFsbDsgd2hpdGUtc3BhY2U6IG5vcm1hbDsgd29yZC13cmFwOiBub3JtYWw7ICI+UXVldWVzPC9kaXY+PC9kaXY+PC9kaXY+PC9mb3JlaWduT2JqZWN0Pjx0ZXh0IHg9IjUzMCIgeT0iMjQ5IiBmaWxsPSIjMDAwMDAwIiBmb250LWZhbWlseT0iSGVsdmV0aWNhIiBmb250LXNpemU9IjEycHgiIHRleHQtYW5jaG9yPSJtaWRkbGUiPlF1ZXVlczwvdGV4dD48L3N3aXRjaD48L2c+PHJlY3QgeD0iMTUwIiB5PSIxMzUiIHdpZHRoPSI0MCIgaGVpZ2h0PSIyMCIgZmlsbD0ibm9uZSIgc3Ryb2tlPSJub25lIiBwb2ludGVyLWV2ZW50cz0iYWxsIi8+PGcgdHJhbnNmb3JtPSJ0cmFuc2xhdGUoLTAuNSAtMC41KSI+PHN3aXRjaD48Zm9yZWlnbk9iamVjdCBzdHlsZT0ib3ZlcmZsb3c6IHZpc2libGU7IHRleHQtYWxpZ246IGxlZnQ7IiBwb2ludGVyLWV2ZW50cz0ibm9uZSIgd2lkdGg9IjEwMCUiIGhlaWdodD0iMTAwJSIgcmVxdWlyZWRGZWF0dXJlcz0iaHR0cDovL3d3dy53My5vcmcvVFIvU1ZHMTEvZmVhdHVyZSNFeHRlbnNpYmlsaXR5Ij48ZGl2IHhtbG5zPSJodHRwOi8vd3d3LnczLm9yZy8xOTk5L3hodG1sIiBzdHlsZT0iZGlzcGxheTogZmxleDsgYWxpZ24taXRlbXM6IHVuc2FmZSBjZW50ZXI7IGp1c3RpZnktY29udGVudDogdW5zYWZlIGNlbnRlcjsgd2lkdGg6IDM4cHg7IGhlaWdodDogMXB4OyBwYWRkaW5nLXRvcDogMTQ1cHg7IG1hcmdpbi1sZWZ0OiAxNTFweDsiPjxkaXYgc3R5bGU9ImJveC1zaXppbmc6IGJvcmRlci1ib3g7IGZvbnQtc2l6ZTogMDsgdGV4dC1hbGlnbjogY2VudGVyOyAiPjxkaXYgc3R5bGU9ImRpc3BsYXk6IGlubGluZS1ibG9jazsgZm9udC1zaXplOiAxMnB4OyBmb250LWZhbWlseTogSGVsdmV0aWNhOyBjb2xvcjogIzAwMDAwMDsgbGluZS1oZWlnaHQ6IDEuMjsgcG9pbnRlci1ldmVudHM6IGFsbDsgd2hpdGUtc3BhY2U6IG5vcm1hbDsgd29yZC13cmFwOiBub3JtYWw7ICI+Q29ubmVjdGlvbjwvZGl2PjwvZGl2PjwvZGl2PjwvZm9yZWlnbk9iamVjdD48dGV4dCB4PSIxNzAiIHk9IjE0OSIgZmlsbD0iIzAwMDAwMCIgZm9udC1mYW1pbHk9IkhlbHZldGljYSIgZm9udC1zaXplPSIxMnB4IiB0ZXh0LWFuY2hvcj0ibWlkZGxlIj5Db25uZWN0Li4uPC90ZXh0Pjwvc3dpdGNoPjwvZz48cmVjdCB4PSI2MzAiIHk9IjEzNSIgd2lkdGg9IjQwIiBoZWlnaHQ9IjIwIiBmaWxsPSJub25lIiBzdHJva2U9Im5vbmUiIHBvaW50ZXItZXZlbnRzPSJhbGwiLz48ZyB0cmFuc2Zvcm09InRyYW5zbGF0ZSgtMC41IC0wLjUpIj48c3dpdGNoPjxmb3JlaWduT2JqZWN0IHN0eWxlPSJvdmVyZmxvdzogdmlzaWJsZTsgdGV4dC1hbGlnbjogbGVmdDsiIHBvaW50ZXItZXZlbnRzPSJub25lIiB3aWR0aD0iMTAwJSIgaGVpZ2h0PSIxMDAlIiByZXF1aXJlZEZlYXR1cmVzPSJodHRwOi8vd3d3LnczLm9yZy9UUi9TVkcxMS9mZWF0dXJlI0V4dGVuc2liaWxpdHkiPjxkaXYgeG1sbnM9Imh0dHA6Ly93d3cudzMub3JnLzE5OTkveGh0bWwiIHN0eWxlPSJkaXNwbGF5OiBmbGV4OyBhbGlnbi1pdGVtczogdW5zYWZlIGNlbnRlcjsganVzdGlmeS1jb250ZW50OiB1bnNhZmUgY2VudGVyOyB3aWR0aDogMzhweDsgaGVpZ2h0OiAxcHg7IHBhZGRpbmctdG9wOiAxNDVweDsgbWFyZ2luLWxlZnQ6IDYzMXB4OyI+PGRpdiBzdHlsZT0iYm94LXNpemluZzogYm9yZGVyLWJveDsgZm9udC1zaXplOiAwOyB0ZXh0LWFsaWduOiBjZW50ZXI7ICI+PGRpdiBzdHlsZT0iZGlzcGxheTogaW5saW5lLWJsb2NrOyBmb250LXNpemU6IDEycHg7IGZvbnQtZmFtaWx5OiBIZWx2ZXRpY2E7IGNvbG9yOiAjMDAwMDAwOyBsaW5lLWhlaWdodDogMS4yOyBwb2ludGVyLWV2ZW50czogYWxsOyB3aGl0ZS1zcGFjZTogbm9ybWFsOyB3b3JkLXdyYXA6IG5vcm1hbDsgIj5Db25uZWN0aW9uPC9kaXY+PC9kaXY+PC9kaXY+PC9mb3JlaWduT2JqZWN0Pjx0ZXh0IHg9IjY1MCIgeT0iMTQ5IiBmaWxsPSIjMDAwMDAwIiBmb250LWZhbWlseT0iSGVsdmV0aWNhIiBmb250LXNpemU9IjEycHgiIHRleHQtYW5jaG9yPSJtaWRkbGUiPkNvbm5lY3QuLi48L3RleHQ+PC9zd2l0Y2g+PC9nPjwvZz48c3dpdGNoPjxnIHJlcXVpcmVkRmVhdHVyZXM9Imh0dHA6Ly93d3cudzMub3JnL1RSL1NWRzExL2ZlYXR1cmUjRXh0ZW5zaWJpbGl0eSIvPjxhIHRyYW5zZm9ybT0idHJhbnNsYXRlKDAsLTUpIiB4bGluazpocmVmPSJodHRwczovL3d3dy5kaWFncmFtcy5uZXQvZG9jL2ZhcS9zdmctZXhwb3J0LXRleHQtcHJvYmxlbXMiIHRhcmdldD0iX2JsYW5rIj48dGV4dCB0ZXh0LWFuY2hvcj0ibWlkZGxlIiBmb250LXNpemU9IjEwcHgiIHg9IjUwJSIgeT0iMTAwJSI+Vmlld2VyIGRvZXMgbm90IHN1cHBvcnQgZnVsbCBTVkcgMS4xPC90ZXh0PjwvYT48L3N3aXRjaD48L3N2Zz4=' />
</center>

Zaletami stosowania architektury, w której komunikacja obsługiwana jest za pomocą kolejki:

- luźne powiązania między obiektami (decoupling),
- łatwa skalowalność ograniczona jedynie do przepustowości samej kolejki wystarczy dodać więcej procesów subskrybentów,
- łączenie wielu języków programowania i platform w ramach jednego systemu,
- większa efektywność niż *HTTP*.

## Uruchomienie RabbitMQ

In [2]:
!docker run --rm -d -p 15672:15672 -p 5672:5672 --name aoop-rabbitmq rabbitmq:3-management

Error: (1,9): error CS1002: ; expected
(1,15): error CS1002: ; expected
(1,24): error CS1002: ; expected
(1,29): error CS1002: ; expected
(1,29): error CS7017: Member definition, statement, or end-of-file expected
(1,39): error CS1002: ; expected
(1,43): error CS1002: ; expected
(1,43): error CS7017: Member definition, statement, or end-of-file expected
(1,51): error CS1002: ; expected
(1,60): error CS1002: ; expected
(1,70): error CS1002: ; expected

Polecenie `docker run` uruchamia kontener z obrazem `rabbitmq:3-management` (jeśli go nie ma zostanie pobrany). Parametr `--rm` powoduje posprzątanie po kontenerze kiedy zostanie wyłączony, `-d` oznacza uruchomienie kontenera jako zadanie w tle, `--name` nadaje nazwę naszemu kontenerowi, a `-p` oznacza mapowanie portu hosta (port używany w przeglądarce / aplikacji zewnętrznej) do portów wewnątrz kontenera. Pierwszy port `15672` będzie używany przez wizualnego managera *RabbitMQ*, a dostęp do *API* za pomocą `5672` (`localhost:5672`). Domyślny login i hasło to: `guest`, `guest`, co można zmienić za pomocą przekazania odpowiedniej zmiennej środowiskowej do polecenia `docker run`. Po wejściu na adres `http://localhost:15672` powinna pokazać się domyślna strona konsoli administracyjnej *RabiitMQ*.

## Wysyłanie i odbieranie wiadomości

Zanim zostanie omówiony sposób wysyłania wiadomości, należy wyszczególnić podstawowe informacje związane z konfiguracją *RabbitMQ*.

Wiadomość składa się z następujących elementów:
- `routing key` - pojedyncze bądź wielokrotne słowo używane do trasowania wiadomości do kolejki,
- `headers` - kolekcja typu klucz wartość, która podobnie jak w przypadku `routing key` może być używana do trasowania wiadomości.
- `payload` - właściwe informacje, które subskrybent wymaga do przeprocesowania wiadomości.
- `delivery mode` może przyjąć postać:
    - `persistent` - wiadomość będzie serializowana na dysk i w razie awarii odtworzona z niego,
    - `transient` - jest przechowana w pamięci,
- `priority` - priorytet wiadomości, użyteczny w kontekście bardzo rozbudowanego systemu, gdzie zdarzenia nie są obsługiwane w czasie rzeczywistym.

Wiadomości nie są przesyłane przez producenta bezpośrednio do kolejki. Za ten element odpowiada *router* (*Exchange*), który może być typu:
- `fanout` - wysyła wiadomość do wszystkich powiązanych kolejek (bez filtracji i routowania),
- `direct` wysyła do pierwszej kolejki na podstawie `routing key`, często jako przykład podaje się wysyłanie logów w zależności od statusu `error`, `warning`, `info`. Dla jednego `Exchange` dodanych jest wiele kolejek, które łączone są na podstawie `routing key` przechowujący jeden ze statusów,
- `topic` działa podobnie jak `direct`, z tą różnicą, że umożlwia przekazywanie do `aggregate key` wildcards,
- `headers`

Najpierw należy zainstalować zależności.

In [1]:
#r "nuget: RabbitMQ.Client, 6.2.1"

Zakładając, że `docker` container działa, domyślną konfiguracją będzie poniższa. Funkcja tworzy fizyczne połączenie z usługą *RabbitMQ*.

In [ ]:
using RabbitMQ.Client;

public IConnection CreateConnection()
{
    ConnectionFactory factory = new ConnectionFactory();
    factory.HostName = "localhost";
    factory.VirtualHost = "/";
    factory.Port = 5672;
    factory.UserName = "guest";
    factory.Password = "guest";
    return factory.CreateConnection();
}

W kodzie zostaną zdefiniowane dwie komórki, jedna do wysyłania, druga do odbierania wiadomości. Kod będzie w ten jednym module, co ma jedynie uzasadnienie w przypadku Jupyter Notebooka. W przypadku normalnej aplikacji powinny być to dwa procesy. Każdy typ trasowania zostanie opisany na przykładzie.

### Trasowanie jako `Fanout`

Najpierw należy zdefiniować `Exchange` (router) i `Queue`. Jeśli, nie istnieją *RabbitMQ* je utworzy, jeśli istnieją zostanie zwrócony do nich uchwyt.


In [3]:
public IModel SetupChannelFanout(IConnection conn)
{
    IModel channel = conn.CreateModel();
    
    channel.ExchangeDeclare(exchange: "aoop-exchange-fanout", 
                            type: "fanout", 
                            durable: true,                             
                            autoDelete: false, 
                            arguments: null);

    channel.QueueDeclare(queue: "aoop-queue-fanout",                          
                         durable: true, 
                         exclusive: false,
                         autoDelete: false, 
                         arguments: null);

    channel.QueueBind("aoop-queue-fanout", "aoop-exchange-fanout", "");
    
    return channel;
}

Typ został zadeklarowany w `type`, z kolei `durable` oznacza tutaj fizyczną reprezentację wiadomości na dysku, dzięki czemu w przypadku restartu usługi wiadomość zostanie dostarczona. Parametr `autoDelete` ustawiony na `True` powoduje usunięcie `Exchange` / `Queue` w momencie, gdy nie będzie podłączonych żadnych producentów / konsumentów wiadomości. Parametr `exclusive` oznacza , że kolejka utworzona jest jednorozowo. 
Niemniej jednak, jeśli zachodzi potrzeba ręcznego usunięcia utworzonych obiektów, można to zróbić na wiele sposób. Najprostszym jest wywołanie funkcji `CleanUpChannel`.

In [4]:
public void CleanUpChannel(IModel channel)
{
    channel.QueueDelete("aoop-queue");    
    channel.ExchangeDelete("aoop-exchange");
}

Następne komórki zawierają kod dodania wiadomości (implementacja *publisher*).

In [5]:
int msgId = 0;

In [1]:
using System.Text;

using (IConnection conn = CreateConnection())
{
    using (IModel channel = SetupChannelFanout(conn))         
    {
        var msg = $"Message {++msgId}";
        
        channel.BasicPublish("aoop-exchange","", null, Encoding.UTF8.GetBytes(msg));
        
        Console.WriteLine($"Message sended: {msg}");            
    }
}

Error: (3,8): error CS0246: The type or namespace name 'IConnection' could not be found (are you missing a using directive or an assembly reference?)
(3,27): error CS0103: The name 'CreateConnection' does not exist in the current context
(5,12): error CS0246: The type or namespace name 'IModel' could not be found (are you missing a using directive or an assembly reference?)
(5,29): error CS0103: The name 'SetupChannelFanout' does not exist in the current context
(7,32): error CS0103: The name 'msgId' does not exist in the current context

Słowa kluczowe `using` są przydatne, gdyż mamy pewność, że zarówno kanał jak i fizyczne połączenie będą w zamknięte przed zakończeniem programu. Wiadomość musi być zamieniona na sekwencje bajtów (binary serialization), co umożliwia przesyłanie dowolnego formatu danych, przed ich serializacją. Kolejna komórka odbiera utworzone wiadomości (implementacja konsumenta).

In [21]:
using System.Text;
using RabbitMQ.Client.Events;
using System.Threading;

using (IConnection conn = CreateConnection())
{
    using (IModel channel = SetupChannelFanout(conn))         
    {
        var consumer = new EventingBasicConsumer(channel);
        consumer.Received += (s,e) => {
            var body = e.Body.ToArray();
            
            string message = Encoding.UTF8.GetString(body);
            
            Console.WriteLine($"Message: {message} from {e.DeliveryTag}");

            //channel.BasicAck(e.DeliveryTag, false); if autoAck = false in BasicConsume
            
            //if error call BasicNack();                                  
        };

        var consumerTag = channel.BasicConsume(queue: "aoop-queue", 
                                               autoAck: true, 
                                               consumer: consumer);          
    }
}

Po przyjęciu i poprawnym obsłużeniu wiadomości należy przekazać komunikat *ack* do usługi *RabbitMQ*. W przeciwnym przypadku ten sam komunikat zostanie przekazany z powrotem do kolejki. Można to zrobić na dwa sposoby ręcznie lub za pomocą flagi `autoAck`.

### Trasowanie jako `Direct`

Wiadomość trasowana jest na podstawie `aggregate key`. Poniżej znajduje się przykład konfiguracji.

In [23]:
public IModel SetupChannelDirect(IConnection conn)
{
    IModel channel = conn.CreateModel();
    
    channel.ExchangeDeclare(exchange: "aoop-exchange-direct", 
                            type: "direct", 
                            durable: true,                             
                            autoDelete: false, 
                            arguments: null);

    channel.QueueDeclare(queue: "aoop-queue-direct-info",                          
                         durable: true, 
                         exclusive: false,
                         autoDelete: false, 
                         arguments: null);
                         
    channel.QueueDeclare(queue: "aoop-queue-direct-warning",                          
                         durable: true, 
                         exclusive: false,
                         autoDelete: false, 
                         arguments: null);
                         
    channel.QueueDeclare(queue: "aoop-queue-direct-error",                          
                         durable: true, 
                         exclusive: false,
                         autoDelete: false, 
                         arguments: null);                             
    
    channel.QueueBind("aoop-queue-direct-info", "aoop-exchange-direct", "info");
    channel.QueueBind("aoop-queue-direct-warning", "aoop-exchange-direct", "warning");
    channel.QueueBind("aoop-queue-direct-error", "aoop-exchange-direct", "error");
    
    return channel;
}        

Połączenie `Queue` i `Exchange` następuje przy użyciu `QueueBind`. Dodatkowo zdefiniowany jest tam `aggregate key`, który używany jest do trasowania.

In [36]:
using System.Text;

using (IConnection conn = CreateConnection())
{
    using (IModel channel = SetupChannelDirect(conn))         
    {
        var msg = $"Message {++msgId}";
        
        channel.BasicPublish("aoop-exchange-direct", 
                             "info",//aggregate key 
                             null, 
                             Encoding.UTF8.GetBytes($"Info: {msg}"));

        channel.BasicPublish("aoop-exchange-direct", 
                             "warning", //aggregate key
                             null, 
                             Encoding.UTF8.GetBytes($"Warning: {msg}"));

        channel.BasicPublish("aoop-exchange-direct", 
                             "error", //aggregate key
                             null, 
                             Encoding.UTF8.GetBytes($"Error: {msg}"));
        
        Console.WriteLine($"Messages sended: {msg}");            
    }
}

Następny kod zawiera implementacje *subscriber*.

In [37]:
using System.Text;
using RabbitMQ.Client.Events;
using System.Threading;

using (IConnection conn = CreateConnection())
{
    using (IModel channel = SetupChannelDirect(conn))         
    {
        var consumer = new EventingBasicConsumer(channel);
        consumer.Received += (s,e) => {
            var body = e.Body.ToArray();
            
            string message = Encoding.UTF8.GetString(body);
            
            Console.WriteLine($"Message: {message} from {e.DeliveryTag}");                    
        };

        channel.BasicConsume(queue: "aoop-queue-direct-info", 
                                               autoAck: true, 
                                               consumer: consumer); 
                                               
        channel.BasicConsume(queue: "aoop-queue-direct-warning", 
                                               autoAck: true, 
                                               consumer: consumer); 
                                               
        channel.BasicConsume(queue: "aoop-queue-direct-error", 
                                               autoAck: true, 
                                               consumer: consumer); 
    }
}

### Trasowanie `Topic`

Działa podobnie jak `Direct` jednak ma więcej możliwość. Do konfiguracji na połączeniu `Queue` i `Exchange` w `aggregate key` można przekazać różne wzorce.

* Asterisk `*` umożlwia podstawienie dokładnie jednego dowolnego klucza agregującego. Przykładowo `*.images` będzie pasował do kolejki `converting.images`, `delete.images`,
* Hash `#` jest bardziej ogólny i zastępuje jedno lub więcej słów np. `#.images` pasuje do `png.converting.images`, `jpg.convering.images` oraz `converting.images`. 

Dodatkowo należy pamiętać, że konfiguracja ta jest na poziomie trasowania, nie konkretnej wiadomości. Poniżej znajduje się przykład użycia tego typu trasowania.

In [51]:
public IModel SetupChannelTopic(IConnection conn)
{
    IModel channel = conn.CreateModel();
    
    channel.ExchangeDeclare(exchange: "aoop-exchange-topic", 
                            type: "topic", 
                            durable: true,                             
                            autoDelete: false, 
                            arguments: null);

    channel.QueueDeclare(queue: "aoop-queue-topic-q1",
                         durable: true, 
                         exclusive: false,
                         autoDelete: false, 
                         arguments: null);
                         
    channel.QueueDeclare(queue: "aoop-queue-topic-q2",                          
                         durable: true, 
                         exclusive: false,
                         autoDelete: false, 
                         arguments: null);
                         
    channel.QueueDeclare(queue: "aoop-queue-topic-q3",                          
                         durable: true, 
                         exclusive: false,
                         autoDelete: false, 
                         arguments: null);                             
    
    channel.QueueBind("aoop-queue-topic-q1", "aoop-exchange-topic", "aoop-queue-topic.*");
    channel.QueueBind("aoop-queue-topic-q2", "aoop-exchange-topic", "aoop-queue-topic.conv.bmp");
    channel.QueueBind("aoop-queue-topic-q3", "aoop-exchange-topic", "aoop-queue-topic.#");
    
    return channel;
}        

In [52]:
using System.Text;

using (IConnection conn = CreateConnection())
{
    using (IModel channel = SetupChannelTopic(conn))         
    {
        var msg = $"Message {++msgId}";
        
        channel.BasicPublish("aoop-exchange-topic", 
                             "aoop-queue-topic.push",//aggregate key 
                             null, 
                             Encoding.UTF8.GetBytes($"Push operation: {msg}"));

        channel.BasicPublish("aoop-exchange-topic", 
                             "aoop-queue-topic.conv.bmp", //aggregate key
                             null, 
                             Encoding.UTF8.GetBytes($"Bitmap as: {msg}"));
        
        Console.WriteLine($"Messages sended: {msg}");            
    }
}

In [53]:
using System.Text;
using RabbitMQ.Client.Events;
using System.Threading;

using (IConnection conn = CreateConnection())
{
    using (IModel channel = SetupChannelTopic(conn))         
    {
        var consumer = new EventingBasicConsumer(channel);
        consumer.Received += (s,e) => {
            var body = e.Body.ToArray();
            
            string message = Encoding.UTF8.GetString(body);
            
            Console.WriteLine($"Message: {message} from {e.DeliveryTag}");                    
        };

        channel.BasicConsume(queue: "aoop-queue-topic-q1", 
                                               autoAck: true, 
                                               consumer: consumer); 
                                               
        channel.BasicConsume(queue: "aoop-queue-topic-q2", 
                                               autoAck: true, 
                                               consumer: consumer); 
                                               
        channel.BasicConsume(queue: "aoop-queue-topic-q3", 
                                               autoAck: true, 
                                               consumer: consumer); 
    }
}

Każda z dwóch wiadomości została wysłana na dwie kolejki. Wzorzec `aoop-queue-topic.#` pasuje do wszystkich `aggregate key`.

### Trasowanie `Headers`

Umożliwia wybieranie kolejki na podstawie nagłówka (w wiadomości przekazywane jako `arguments`). Dodatkowo binding może zawierać konfigurację `x-match` w swojej definicji w dwóch wersjach `all` lub `any`. Pierwsza wersja oznacza, że każdy atrybut musi się zgadzać między wiadomością, a połączeniem `Queue` i `Exchange`. Z kolei `any` oznacza, że co najmniej jeden nagłówek musi pasować. Ilustruje to poniższy przykład.

In [55]:
public IModel SetupChannelTopic(IConnection conn)
{
    IModel channel = conn.CreateModel();
    
    channel.ExchangeDeclare(exchange: "aoop-exchange-headers", 
                            type: "headers", 
                            durable: true,                             
                            autoDelete: false, 
                            arguments: null);

    channel.QueueDeclare(queue: "aoop-queue-headers-q1",
                         durable: true, 
                         exclusive: false,
                         autoDelete: false, 
                         arguments: null);
                         
    channel.QueueDeclare(queue: "aoop-queue-headers-q2",                          
                         durable: true, 
                         exclusive: false,
                         autoDelete: false, 
                         arguments: null);        
    
    channel.QueueBind("aoop-queue-headers-q1", "aoop-exchange-headers", "", new Dictionary<string, object>()
                                                                            {
                                                                                {"x-match","all" },
                                                                                {"job","convert" },
                                                                                {"format","jpeg" }
                                                                            });
                                                                            
    channel.QueueBind("aoop-queue-headers-q2", "aoop-exchange-headers", "", new Dictionary<string, object>()
                                                                            {
                                                                                {"x-match","any" },
                                                                                {"job","convert" },
                                                                                {"format","jpeg" }
                                                                            });                                                                                
    return channel;
}     

Ostatni argument funkcji `QueueBind` przyjmuje listę atrybutów związanych z trasowaniem wiadomości do konkretnej kolejki. Pierwszy binding oznacza trasowanie występowania w wiadomości wszystkich pól w `headers`. Kolejny przykład ilustruje w jaki sposób wysyłać i odbierać tego typu wiadomości.

In [56]:
using System.Text;

using (IConnection conn = CreateConnection())
{
    using (IModel channel = SetupChannelTopic(conn))         
    {
        var msg = $"Message {++msgId}";
        
        IBasicProperties props = channel.CreateBasicProperties();
        props.Headers = new Dictionary<string, object>();
        props.Headers.Add("job", "convert");
        props.Headers.Add("format", "jpeg");
        
        channel.BasicPublish("aoop-exchange-headers", 
                             "",//aggregate key 
                             props, 
                             Encoding.UTF8.GetBytes($"Conversion operation jpeg: {msg}"));
                             
        props = channel.CreateBasicProperties();
        props.Headers = new Dictionary<string, object>();
        props.Headers.Add("job", "convert");
        props.Headers.Add("format", "bmp");                             

        channel.BasicPublish("aoop-exchange-headers", 
                             "", //aggregate key
                             props, 
                             Encoding.UTF8.GetBytes($"Conversion operation bmp: {msg}"));                                            
        
        Console.WriteLine($"Messages sended: {msg}");            
    }
}

Należy zauważyć, że `aggregate key` jest pusty z powodu tego, że nie będziemy trasować wiadomości na jego podstawie. Kolejny przykład odbiera wiadomość.

In [57]:
using System.Text;
using RabbitMQ.Client.Events;
using System.Threading;

using (IConnection conn = CreateConnection())
{
    using (IModel channel = SetupChannelTopic(conn))         
    {
        var consumer = new EventingBasicConsumer(channel);
        consumer.Received += (s,e) => {
            var body = e.Body.ToArray();
            
            string message = Encoding.UTF8.GetString(body);
            
            Console.WriteLine($"Message: {message} from {e.DeliveryTag}");                    
        };

        channel.BasicConsume(queue: "aoop-queue-headers-q1", 
                                               autoAck: true, 
                                               consumer: consumer); 
                                               
        channel.BasicConsume(queue: "aoop-queue-headers-q2", 
                                               autoAck: true, 
                                               consumer: consumer);                                            
    }
}

Pierwsza wiadomość pasuje zarówno do warunku `x-match` `any` oraz  nagłówków `convert` i `jpeg`.

## Event Sourcing

Jeśli przyjąć, że zamiast wiadomości system kolejkowy używany jest do wysyłania i odbierania zdarzeń w systemie otrzymuje się tzw. *Event driven architecture*. Jej zaletą jest dobra skalowalność. Często spotykaną praktyką jest dodanie do tej architektury kolejnej warstwy *event sourcing*. Polega ona na tym, że każde zdarzenie jest przechowywane w bazie danych. Zaletą takiego podejścia jest to, że dostępny jest cały log wszystkich operacji, a co ważne w przypadku awari łatwo odtworzyć od zera system. Z *RabbitMQ* wystarczy dodać ścieżkę dla każdego zdarzenia, aby jeden worker kolekcjonował zdarzenia (zapisywał je do bazy danych).


## Zadania do wykonania

Zadanie 1. W środowisku *microservice* są dwa serwisy: `Order Service` i `Authentication Service`. Każdy z nich musi wysyłać maila do użytkownika z innego powodu. Pierwszy, aby poinformować użytkownika o zmianie jego status zamówienia, drugi aby wysłać mu przypomnienie hasła. Dane są następujące założenia:

1. Wiadomości nie mogą zaginąć nawet, jeśli usługa zostanie zrestartowana.
2. Każda wiadomość musi być przetworzona dokładnie jeden raz.
3. Jeśli usługa wysyłająca maile zwróci błąd, wiadomość powinna trafić z powrotem do kolejki.
4. Może być wiele *microservice* do wysyłania wiadomości.